# Comptime Demo

This notebook showcases the new experimental comptime mode in Guppy.

In [1]:
from guppylang import GuppyModule, qubit
from guppylang.decorator import guppy
from guppylang.std.builtins import array
from guppylang.std.quantum import h, cx, rz, measure
from guppylang.std.angles import angle, pi
from guppylang.std import quantum
from guppylang.std import angles

from hugr.hugr.render import DotRenderer

## Intro

Comptime functions are executed using the Python interpreter which drives the Hugr generation. Thus, comptime functions can contain arbitrary Python code, but everything is evaluated at compile-time. The result is a flat Hugr program:

In [2]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.comptime(module)
def ladder() -> array[qubit, 5]:
    qs = [qubit() for _ in range(5)]
    for q in qs:
        h(q)
    for q1, q2 in zip(qs[1:], qs[:-1]):
        cx(q1, q2)
    return qs

DotRenderer().render(module.compile().module);

Comptime functions can be called from regular Guppy functions and vice versa.

In [3]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy(module)
def regular1() -> tuple[qubit, qubit]:
    q1 = qubit()
    h(q1)
    q2 = comptime_func(q1)
    return q1, q2

@guppy.comptime(module)
def comptime_func(q: qubit) -> qubit:
    r = regular2()
    cx(q, r)
    return r

@guppy(module)
def regular2() -> qubit:
    q = qubit()
    h(q)
    return q

module.compile();

Comptime functions can even call out to non-Guppy functions and pass qubits along as data:

In [4]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.comptime(module)
def foo() -> qubit:
    q = qubit()
    bar(q)
    return q

def bar(q):
    """Regular Python function, no type annotations needed"""
    h(q)

module.compile();

## Arithmetic

Traced functions can do arbitrary arithmetic on their inputs:

In [5]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.comptime(module)
def foo(q: qubit, x: float) -> None:
    x = x * 2
    #print(x)  # What is x?
    rz(q, angle(x))

module.compile();

However, we are not allowed to branch conditioned on the value of `x`:

In [6]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.comptime(module)
def foo(q: qubit, x: float) -> None:
    x = x * 2
    if x > 3:
        rz(q, angle(x))

module.compile();

GuppyComptimeError: Can't branch on a dynamic Guppy value since it's concrete value is not known at comptime. Consider defining a regular Guppy function to perform dynamic branching.

Similarly, we can't branch on measurement results inside the tracing context:

In [7]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.comptime(module)
def foo(q: qubit) -> None:
    if measure(q):
        print("XXX")

module.compile();

GuppyComptimeError: Can't branch on a dynamic Guppy value since it's concrete value is not known at comptime. Consider defining a regular Guppy function to perform dynamic branching.

Also, we can't use regular inputs to control the size of registers:

In [8]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.comptime(module)
def foo(n: int) -> None:
    qs = [qubit() for _ in range(n)]

module.compile();

TypeError: 'GuppyObject' object cannot be interpreted as an integer

## Arrays and Lists

Input arrays can be used like regular lists.

In [9]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.declare(module)
def foo(qs: array[qubit, 10]) -> None: ...

@guppy.comptime(module)
def bar(qs: array[qubit, 10]) -> None:
    # Arrays are iterable in the Python context
    for q1, q2 in zip(qs[1:], qs[:-1]):
        cx(q1, q2)
    [start, *_, end] = qs
    cx(start, end)

    # Regular Guppy functions can be called with Python lists
    # as long as the lengths match up
    rev = [q for q in reversed(qs)]
    foo(rev)

module.compile();

## Safety

In [10]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.comptime(module)
def bad(q: qubit) -> None:
    cx(q, q)

module.compile()

GuppyComptimeError: Value with non-copyable type `qubit` was already used

Previous use occurred in <In [10]>:6 as an argument to `cx`

In [11]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.comptime(module)
def bad(q: qubit) -> qubit:
    return q

module.compile()

Error: Error in comptime function return (at <In [11]>:5:0)
  | 
3 | 
4 | @guppy.comptime(module)
5 | def bad(q: qubit) -> qubit:
  | ^^^^^^^^^^^^^^^^^^^^^^^^^^^
6 |     return q
  | ^^^^^^^^^^^^

Argument `q` is borrowed, so it is implicitly returned to the caller. Value with
non-copyable type `qubit` was already used

Previous use occurred in <In [?]>:8.

Guppy compilation failed due to 1 previous error


In [12]:
module = GuppyModule("test")
module.load_all(quantum)

@guppy.comptime(module)
def bad(q: qubit) -> None:
    tmp = qubit()
    cx(tmp, q)

module.compile()

Error: Error in comptime function return (at <In [12]>:5:0)
  | 
3 | 
4 | @guppy.comptime(module)
5 | def bad(q: qubit) -> None:
  | ^^^^^^^^^^^^^^^^^^^^^^^^^^
  | ...
7 |     cx(tmp, q)
  | ^^^^^^^^^^^^^^

Value with non-droppable type `qubit` is leaked by this function

Guppy compilation failed due to 1 previous error
